# Switchpoint simulations

In [ ]:
import numpy as np
import pandas as pd
import os,sys
import random
import time

In [ ]:
def cellDivisions(tm,td,ts,rl):
    global n
    for dividingCell in range(0,n):
        tc = ts - tm[dividingCell]
        tr = td[dividingCell] - tc
        if tr < 1/3600:
            tm[dividingCell] = ts
            tm = np.insert(tm,dividingCell,ts)
            tm = tm[:-1]            
            n = n+1
            td[dividingCell] = np.random.normal(340, 32)
            td = np.insert(td,dividingCell,np.random.normal(340, 32))
            td = td[:-1]            
    return tm,td

In [ ]:
def run(steps,tm,td,rl):
    global n
    outgrowth = []
    for ts in range(0,steps+1,1):
        tm,td = cellDivisions(tm,td,ts,rl)
        outgrowth.append(n-n0)    # Appends the outgrowth for each time
    return outgrowth

In [ ]:
root = '/home/ecuracosta/Desktop/MS/Simulations/Switchpoint_fixing/'
os.chdir(root)

random.seed(33)

start_time = time.time() 

emptySpace = 250
steps = 24*8    # Number of steaps (in hours)
n0_array = np.arange(100,301,10)    # Number of cells after amputation
rl_array = -np.arange(500,1505,50)    # Recruitment limit
tau_array = np.arange(100,301,6)

n_array = []
for n0 in n0_array:
    for rl in rl_array:
        n02 = n0+int(round(rl/13.2))
        lattice = n02+emptySpace
        folder = 'n0='+str(n0)+'_rl='+str(rl)
        os.makedirs(folder)
        print('n0:',n0)
        print('rl:',rl)
        print('-------')
        for seed in range(0,200):
            n = n02
            tm = np.random.uniform(-(340-2*32),0,lattice)
            td = np.random.normal(340,32,lattice)
            solution = run(steps,tm,td,rl)
            d = {"time":np.arange(0,steps+1),"outgrowth":solution}
            df = pd.DataFrame(d)
            outfile = open(str(root)+str(folder)+'/'+'sim_seed='+str(seed)+'.csv', 'a')
            df.to_csv(outfile, sep=',')
            outfile.close()
print("Total time: %s seconds <--" % (time.time() - start_time))